# SH_sample_properties
This notebook analyzes Solid Hydrogen sample properties using ROOT (PyROOT) and RDataFrame.
It loads a ROOT file, creates filtered/derived dataframes, produces histograms and plots (neutrino energy, interaction vertices, transverse momentum, event-type stacks), and writes results to `Plots/SH_sample_properties/SH_sample_properties.root`.

Usage: set the input ROOT file path and output path in the first code cell, run cells sequentially.

Outputs: PDF/PNG plots and a ROOT file with saved histograms.

Dependencies: ROOT (PyROOT) and the local helper module `df_functions_utils.py`.

In [ ]:
import ROOT
from ROOT import TCanvas
from ROOT import TLegend
from ROOT import RDataFrame
from ROOT import kRed, kBlue
import df_functions_utils as df_utils

df = RDataFrame("Rough_tree", "/storage/gpfs_data/neutrino/users/battisti/hydrogen_analysis_tests/Solid_Hydrogen2/Snap_file_processing/Processed_tree.root")
Out_dir = "Plots/SH_sample_properties/"

In [ ]:
### Uncoment for fast testing
df = df.Range(1000)

### Generating output file

In [ ]:
f_out = ROOT.TFile(Out_dir + "SH_sample_properties.root", "RECREATE")

### Evaluating ratio between Carbon events in Plastic/Graphite

In [ ]:
evt_C_pol = df.Filter('Where_int == "C3H6" && target == "tgt:1000060120"').Count().GetValue()
evt_C_gr = df.Filter('Where_int == "Graphite" && target == "tgt:1000060120"').Count().GetValue()

print("Ratio of  C-Graphite events to C-C3H6 events:", evt_C_gr/evt_C_pol)

### Evaluating ratio between Plastic and Graphite events over the total 

In [ ]:
evt_tot = df.Count().GetValue()
evt_pol = df.Filter('Where_int == "C3H6"').Count().GetValue()
evt_gr = df.Filter('Where_int == "Graphite"').Count().GetValue()

print("Total events:", evt_tot)
print("C3H6 events:", evt_pol)
print("Graphite events:", evt_gr)
print("Ratio of C3H6 events to total events:", evt_pol/evt_tot)
print("Ratio of Graphite events to total events:", evt_gr/evt_tot)

# Generating df with additional columns

### List of dataframes:
- df = contains Rough_tree with all its columns
- df_extend =  df + additional columns needed in the analysis 
- df_flag = df_extended + flags that identify the filters 
- df_Tr_mom = contains all the events of the Rough_tree with its columns + columns used to calculate the transverse momentum 

### Filters flags for df_flag:
- inside the STT volume (volume_filter)
- successufull reconstruction (reco_filter)
- one track (track_filter)
- neutron space correspondence (space_filter, done with clusters) (spaceTRUE_filter, done with hit segment starts)
- neutron time correspondence (time_filter, done with clusters) (timeTRUE_filter, done with hit segment starts)
- events CC-QE with antimuon and neutron in the final state, on H (true_filter) 

In [ ]:
df_extended = df_utils.df_extend(df)
df_flag = df_utils.df_addflags(df_extended)
df_Tr_mom = df_utils.df_addtransversemomentum(df)

# Plotting

### True Neutrino energy  

In [ ]:
### Comment to avoid interactivity 
%jsroot

In [ ]:
c1 = ROOT.TCanvas("c", "Canvas", 900, 600)
h_nu = df.Histo1D(("h_nu_energy", "Neutrino Energy", 999, 0, 10000), "Enu")

h_nu.GetXaxis().SetTitle("True Neutrino Energy MC (MeV)")
h_nu.SetLineColor(ROOT.kBlue)
h_nu.Draw("")

c1.Draw()
c1.SaveAs(Out_dir + "nu_energy.pdf")

f_out.cd()
h_nu.GetValue().Write()

In [ ]:
#Logarithmic scale

c1 = ROOT.TCanvas("c", "Canvas", 900, 600)
h_nu_l = df.Histo1D(("h_nu_log", "Neutrino Energy log", 700, 0, 10000), "Enu")

h_nu_l.GetXaxis().SetTitle("True Neutrino Energy MC (MeV)")
c1.SetLogy()

h_nu_l.SetLineColor(ROOT.kBlue)


h_nu_l.Draw("E")

c1.Draw()
c1.SaveAs(Out_dir + "nu_energy_log.pdf")

f_out.cd()
h_nu_l.GetValue().Write()

### Interaction vertexes

In [ ]:
from array import array

h_xy = df.Histo2D(("h_nu_vertices_xy", "", 100, -3000, 3000, 100, -6000, 1000),"vertex_x", "vertex_y")
h_zy = df.Histo2D(("h_nu_verticeszy", "", 100, 20000, 28000, 100, -6000, 1000),"vertex_z", "vertex_y")

c = TCanvas("c1", "Canvas", 1500, 600)
c.Divide(2, 1)
c.Draw()

ROOT.gStyle.SetOptStat(0)

stops = [0.00, 0.33, 0.66, 1.00]
val_r = [1.00, 1.00, 0.95, 0.85]
val_g = [1.00, 0.85, 0.60, 0.40]
val_b = [1.00, 0.60, 0.20, 0.00]

ROOT.TColor.CreateGradientColorTable(
    len(stops),
    array('d', stops),
    array('d', val_r),  # R
    array('d', val_g),  # G
    array('d', val_b),  # B
    255
)



ROOT.gStyle.SetNumberContours(255)


c.cd(1)
pad1 = c.GetPad(1)
pad1.SetLeftMargin(0.15)   # margine sinistro più largo per la y
pad1.SetBottomMargin(0.15) # margine inferiore più largo per la x
h_xy.GetXaxis().SetTitle("x [mm]")
h_xy.GetYaxis().SetTitle("y [mm]")
h_xy.Draw("COL")

c.cd(2)
pad2 = c.GetPad(2)
pad2.SetLeftMargin(0.15)
pad2.SetRightMargin(0.15)
pad2.SetBottomMargin(0.15)
h_zy.GetXaxis().SetTitle("z [mm]")
h_zy.GetYaxis().SetTitle("y [mm]")
h_zy.Draw("COLZ")

c.SaveAs(Out_dir + "nu_vertices_xy_zy.pdf")


f_out.cd()
h_xy.GetValue().Write()
h_zy.GetValue().Write()


In [ ]:
#Events on STT tracker
df_V = df_flag.Filter("volume_flag == 1")

h_xy_V = df_V.Histo2D(("h_nu_vertices_xy_STT", " Interaction vertexes xy STT", 100, -3000, 3000, 100, -6000, 1000),"vertex_x", "vertex_y")
h_zy_V = df_V.Histo2D(("h_nu_vertices_zy_STT", " Interaction vertexes zy STT", 100, 20000, 28000, 100, -6000, 1000),"vertex_z", "vertex_y")

c_v = TCanvas("c1", "Canvas", 1500, 600)
c_v.Divide(2, 1)
c_v.Draw()

c_v.cd(1)
pad1 = c_v.GetPad(1)
pad1.SetLeftMargin(0.15)   # margine sinistro più largo per la y
pad1.SetBottomMargin(0.15) # margine inferiore più largo per la x
h_xy_V.GetXaxis().SetTitle("x [mm]")
h_xy_V.GetYaxis().SetTitle("y [mm]")
h_xy_V.Draw("COLZ")

c_v.cd(2)
pad2 = c_v.GetPad(2)
pad2.SetLeftMargin(0.15)
pad2.SetRightMargin(0.15)
pad2.SetBottomMargin(0.15)
h_zy_V.GetXaxis().SetTitle("z [mm]")
h_zy_V.GetYaxis().SetTitle("y [mm]")
h_zy_V.Draw("COLZ")


c_v.SaveAs(Out_dir + "nu_vertices_xy_zy_STT.pdf")


f_out.cd()
h_xy_V.GetValue().Write()
h_zy_V.GetValue().Write()

In [ ]:
#Events on plastic 

h_xy_Pl = df_V.Filter('Where_int == "C3H6"').Histo2D(("h_nu_vertices_xy_Plastic", " Interaction vertexes xy in C3H6", 100, -3000, 3000, 100, -6000, 1000),"vertex_x", "vertex_y")
h_zy_Pl = df_V.Filter('Where_int == "C3H6"').Histo2D(("h_nu_vertices_zy_Plastic", " Interaction vertexes zy in C3H6", 100, 20000, 28000, 100, -6000, 1000),"vertex_z", "vertex_y")

c_p = TCanvas("c1", "Canvas", 500, 400)


c_p.SetLeftMargin(0.15)
c_p.SetBottomMargin(0.15)
c_p.SetRightMargin(0.15)

h_zy_Pl.GetXaxis().SetTitle("z [mm]")
h_zy_Pl.GetYaxis().SetTitle("y [mm]")
h_zy_Pl.Draw("COLZ")

c_p.Draw()
c_p.SaveAs(Out_dir + "nu_vertices_zy_Plastic.pdf")



f_out.cd()
h_zy_Pl.GetValue().Write()

In [ ]:
#Events on graphite

h_xy_G = df_V.Filter('Where_int == "Graphite"').Histo2D(("h_nu_vertices_xy_Graphite", "Interaction vertexes xy in Graphite", 100, -3000, 3000, 100, -6000, 1000),"vertex_x", "vertex_y")
h_zy_G = df_V.Filter('Where_int == "Graphite"').Histo2D(("h_nu_vertices_zy_Graphite", "Interaction vertexes zy in Graphite", 100, 20000, 28000, 100, -6000, 1000),"vertex_z", "vertex_y")

c_g = TCanvas("c1", "Canvas", 500, 400)

c_g.SetLeftMargin(0.15)
c_g.SetBottomMargin(0.15)
c_g.SetRightMargin(0.15)

h_zy_G.GetXaxis().SetTitle("z [mm]")
h_zy_G.GetYaxis().SetTitle("y [mm]")
h_zy_G.Draw("COLZ")

c_g.Draw()
c_g.SaveAs(Out_dir + "nu_vertices_zy_Graphite.pdf")

f_out.cd()
h_zy_G.GetValue().Write()

### Transverse momentum

In [ ]:

ROOT.gStyle.SetOptStat(0)

h_H_tr = df_Tr_mom.Filter('target == "tgt:1000010010"').Histo1D(("h_H", "Transverse momentum on H ", 1000, -0.05, 0.8), "transverse_mom_GeV")
h_C_tr = df_Tr_mom.Filter('target == "tgt:1000060120"').Histo1D(("h_C", " Transverse momentum on C", 1000, -0.05, 0.8), "transverse_mom_GeV")


c_tr = TCanvas("c_tr", "Canvas", 900, 600)
c_tr.SetLogy()

h_H_tr.SetLineColor(kRed)
h_C_tr.SetLineColor(kBlue)

h_H_tr.GetXaxis().SetTitle("Transverse momentum [MeV]")
h_H_tr.GetYaxis().SetTitle("Entries")
h_H_tr.GetYaxis().SetTitleOffset(0.6)

h_H_tr.Draw("HIST")
h_C_tr.Draw("HIST SAME")


leg = TLegend(0.7, 0.75, 0.9, 0.9)
leg.AddEntry(h_H_tr.GetPtr(), "#bar{#nu}_{#mu} CC on Hydrogen", "l")
leg.AddEntry(h_C_tr.GetPtr(), "#bar{#nu}_{#mu} CC on Carbon", "l")
leg.Draw()

c_tr.Draw()
c_tr.SaveAs(Out_dir + "tr_mom.pdf")

f_out.cd()
h_H_tr.GetValue().Write()
h_C_tr.GetValue().Write()


In [ ]:
### Quick table check
df_Tr_mom.Filter('target == "tgt:1000060120" && transverse_mom_GeV == 0.0').Display(["Where_int", "st_proc_type","primaries_PDG"],50).Print()
del df_Tr_mom

### Nu energy divided by interaction type on Plastic and Graphite

In [ ]:
#Events on plastic 
nbin = 12 
nmin = 0
nmax = 6000


ROOT.gStyle.SetHistLineColor(ROOT.kBlack)

hist_h1 = df_V.Filter("Evt_cat == 1").Histo1D(("hist_1", "", nbin, nmin, nmax),"Enu")
hist_h4 = df_V.Filter('target == "tgt:1000010010" && Where_int == "C3H6" && st_proc_type == "proc:Weak[CC],RES"').Histo1D(("hist_h4", "", nbin, nmin, nmax),"Enu")
hist_h5 = df_V.Filter('target == "tgt:1000010010" && Where_int == "C3H6" && st_proc_type == "proc:Weak[CC],DIS"').Histo1D(("hist_h5", "", nbin, nmin, nmax),"Enu")
hist_c1 = df_V.Filter("Evt_cat == 2").Histo1D(("hist_2", "", nbin, nmin, nmax),"Enu")
hist_c2 = df_V.Filter('target == "tgt:1000060120" && st_proc_type == "proc:Weak[CC],RES" && Where_int == "C3H6"').Histo1D(("hist_h5", "", nbin, nmin, nmax),"Enu")
hist_c3 = df_V.Filter('target == "tgt:1000060120" && st_proc_type == "proc:Weak[CC],DIS" && Where_int == "C3H6"').Histo1D(("hist_h5", "", nbin, nmin, nmax),"Enu")
hist_c4 = df_V.Filter('target == "tgt:1000060120" && st_proc_type == "proc:Weak[CC],MEC" && Where_int == "C3H6"').Histo1D(("hist_h5", "", nbin, nmin, nmax),"Enu")
hist_c5 = df_V.Filter('target == "tgt:1000060120" && st_proc_type == "proc:Weak[CC],COH" && Where_int == "C3H6"').Histo1D(("hist_h5", "", nbin, nmin, nmax),"Enu")

hist_h1.SetFillColor(ROOT.kBlue)
hist_h4.SetFillColor(ROOT.kBlue+2)
hist_h5.SetFillColor(ROOT.kBlue-3)

hist_c1.SetFillColor(ROOT.kRed)
hist_c2.SetFillColor(ROOT.kRed+2)
hist_c3.SetFillColor(ROOT.kOrange)
hist_c4.SetFillColor(ROOT.kOrange+7)
hist_c5.SetFillColor(ROOT.kYellow)

stack = ROOT.THStack("h_nu_energy_int_stack_Plastic", " ")
stack.Add(hist_h1.GetPtr())
stack.Add(hist_h4.GetPtr())
stack.Add(hist_h5.GetPtr())

stack.Add(hist_c1.GetPtr())
stack.Add(hist_c2.GetPtr())
stack.Add(hist_c3.GetPtr())
stack.Add(hist_c4.GetPtr())
stack.Add(hist_c5.GetPtr())


c = ROOT.TCanvas()
c.SetLeftMargin(0.15)

stack.Draw("HIST")
stack.GetXaxis().SetTitle("True Neutrino Energy [MeV]")
stack.GetYaxis().SetTitle("Entries")

legend = ROOT.TLegend(0.62, 0.55, 0.90, 0.90)


legend.AddEntry(hist_h1.GetPtr(), "CC-QE on H in plastic", "f")
legend.AddEntry(hist_h4.GetPtr(), "CC-RES on H in plastic", "f")
legend.AddEntry(hist_h5.GetPtr(), "CC-DIS on H in plastic", "f")

legend.AddEntry(hist_c1.GetPtr(), "CC-QE on C in plastic", "f")
legend.AddEntry(hist_c2.GetPtr(), "CC-RES on C in plastic", "f")
legend.AddEntry(hist_c3.GetPtr(), "CC-DIS on C in plastic", "f")
legend.AddEntry(hist_c4.GetPtr(), "CC-MEC on C in plastic", "f")
legend.AddEntry(hist_c5.GetPtr(), "CC-COH on C in plastic", "f")

legend.Draw()

c.Draw()
c.SaveAs(Out_dir+"plastic_comp.pdf")


f_out.cd()
# hist_h1.GetValue().Write()
# hist_h4.GetValue().Write()
# hist_h5.GetValue().Write()
# hist_c1.GetValue().Write()
# hist_c2.GetValue().Write()
# hist_c3.GetValue().Write()
# hist_c4.GetValue().Write()
# hist_c5.GetValue().Write()

stack.Write()

In [ ]:
#Events on graphite 
nbin = 12
nmin = 0
nmax = 6000


hist_g1 = df_V.Filter("Evt_cat == 4").Histo1D(("hist_4", "", nbin, nmin, nmax),"Enu")
hist_g2 = df_V.Filter('target == "tgt:1000060120" && st_proc_type == "proc:Weak[CC],RES" && Where_int == "Graphite"').Histo1D(("hist_h5", "", nbin, nmin, nmax),"Enu")
hist_g3 = df_V.Filter('target == "tgt:1000060120" && st_proc_type == "proc:Weak[CC],DIS" && Where_int == "Graphite"').Histo1D(("hist_h5", "", nbin,nmin, nmax),"Enu")
hist_g4 = df_V.Filter('target == "tgt:1000060120" && st_proc_type == "proc:Weak[CC],MEC" && Where_int == "Graphite"').Histo1D(("hist_h5", "", nbin,nmin, nmax),"Enu")
hist_g5 = df_V.Filter('target == "tgt:1000060120" && st_proc_type == "proc:Weak[CC],COH" && Where_int == "Graphite"').Histo1D(("hist_h5", "", nbin,nmin, nmax),"Enu")

hist_g1.SetFillColor(ROOT.kGreen+1)
hist_g2.SetFillColor(ROOT.kGreen+3)
hist_g3.SetFillColor(ROOT.kGreen-2)
hist_g4.SetFillColor(ROOT.kGreen-5)
hist_g5.SetFillColor(ROOT.kGreen-10)

stack2 = ROOT.THStack("h_nu_energy_int_stack_Graphite", " ")
stack2.Add(hist_g1.GetPtr())
stack2.Add(hist_g2.GetPtr())
stack2.Add(hist_g3.GetPtr())
stack2.Add(hist_g4.GetPtr())
stack2.Add(hist_g5.GetPtr())

c = ROOT.TCanvas()
c.SetLeftMargin(0.15)

stack2.Draw("HIST")
stack2.GetXaxis().SetTitle("True Neutrino Energy [MeV]")
stack2.GetYaxis().SetTitle("Entries")

legend = ROOT.TLegend(0.62, 0.55, 0.90, 0.90)
legend.AddEntry(hist_g1.GetPtr(), "CC-QE on C in graphite", "f")
legend.AddEntry(hist_g2.GetPtr(), "CC-RES on C in graphite", "f")
legend.AddEntry(hist_g3.GetPtr(), "CC-DIS on C in graphite", "f")
legend.AddEntry(hist_g4.GetPtr(), "CC-MEC on C in graphite", "f")
legend.AddEntry(hist_g5.GetPtr(), "CC-COH on C in graphite", "f")

legend.Draw()

c.Draw()
c.SaveAs(Out_dir+"graph.pdf")


f_out.cd()
# hist_g1.GetValue().Write()
# hist_g2.GetValue().Write()
# hist_g3.GetValue().Write()
# hist_g4.GetValue().Write()
# hist_g5.GetValue().Write()

stack2.Write()



# Closing output file

In [ ]:
f_out.Close()